In [ ]:
import os
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
DB_FAISS_PATH = "vectorstore/db_faiss"

@st.cache_resource
def load_vectorstore():
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

In [ ]:
def load_llm(repo_id, token):
    return HuggingFaceEndpoint(
        repo_id=repo_id,
        temperature=0.5,
        model_kwargs={"token": token, "max_length": "512"}
    )

In [ ]:
def set_custom_prompt(template):
    return PromptTemplate(template=template, input_variables=["context", "question"])

def main():
    st.title("Interactive Chatbot")
    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        st.chat_message(message["role"]).markdown(message["content"])

    user_input = st.chat_input("Enter your query")
    if user_input:
        st.chat_message("user").markdown(user_input)
        st.session_state.messages.append({"role": "user", "content": user_input})

        custom_prompt_template = """
        Use the pieces of information provided in the context to answer the user's question.
        If you don't know the answer, just say so. Do not make up answers or include anything not from the context.

        Context: {context}
        Question: {question}
        """

        HUGGINGFACE_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.3"
        HF_TOKEN = os.environ.get("HF_TOKEN")

        try:
            vectorstore = load_vectorstore()
            qa_chain = RetrievalQA.from_chain_type(
                llm=load_llm(HUGGINGFACE_REPO_ID, HF_TOKEN),
                chain_type="stuff",
                retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
                return_source_documents=True,
                chain_type_kwargs={"prompt": set_custom_prompt(custom_prompt_template)}
            )
            response = qa_chain.invoke({"query": user_input})
            result = response["result"]
            sources = response["source_documents"]
            output = f"{result}\n\nSource Documents:\n{sources}"
            st.chat_message("assistant").markdown(output)
            st.session_state.messages.append({"role": "assistant", "content": output})
        except Exception as e:
            st.error(f"Error: {e}")

if __name__ == "__main__":
    main()